# Matrix Risk Engine - Risk Dashboard Notebook

This notebook demonstrates the risk analytics workflow:
1. Load portfolio positions
2. Calculate VaR and CVaR
3. Compute Greeks
4. Run stress tests

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

# Add project root to path
import sys
sys.path.insert(0, '..')

## 1. Define Portfolio

In [ ]:
from src.core.domain.portfolio import Portfolio, PortfolioMetadata

# Create sample portfolio
portfolio = Portfolio(
    positions={
        'AAPL': 1000,
        'MSFT': 500,
        'GOOGL': 200,
        'AMZN': 100,
        'META': 300,
    },
    weights={
        'AAPL': 0.35,
        'MSFT': 0.25,
        'GOOGL': 0.20,
        'AMZN': 0.10,
        'META': 0.10,
    },
    nav=1000000.0,
    as_of_date=date.today(),
    metadata=PortfolioMetadata(strategy_name='Tech Portfolio'),
)

print(f"Portfolio NAV: ${portfolio.nav:,.0f}")
print(f"\nPositions:")
for symbol, weight in portfolio.weights.items():
    print(f"  {symbol}: {weight:.1%}")

## 2. Load Market Data

In [ ]:
# Generate sample market data (2 years for VaR calculation)
np.random.seed(42)
dates = pd.date_range('2022-01-01', periods=504, freq='B')

market_data = {}
for symbol in portfolio.weights:
    returns = np.random.randn(504) * 0.02
    prices = 100 * np.cumprod(1 + returns)
    market_data[symbol] = prices

prices_df = pd.DataFrame(market_data, index=dates)
print(f"Market data: {len(prices_df)} days")
prices_df.tail()

## 3. Calculate VaR and CVaR

In [ ]:
from src.adapters.ore_adapter import OREAdapter

# Create risk adapter
risk_adapter = OREAdapter()

# Calculate Historical VaR
var_results = risk_adapter.calculate_var(
    portfolio=portfolio,
    market_data=prices_df,
    method='historical',
    confidence_levels=[0.95, 0.99],
    window_days=250,
)

# Calculate CVaR
cvar_results = risk_adapter.calculate_cvar(
    portfolio=portfolio,
    market_data=prices_df,
    var_params={
        'method': 'historical',
        'confidence_levels': [0.95, 0.99],
        'window_days': 250,
    },
)

print("Value at Risk (VaR)")
print("=" * 40)
for level, value in var_results.items():
    print(f"{level} VaR: ${abs(value):,.0f} ({abs(value/portfolio.nav)*100:.2f}%)")

print("\nConditional VaR (Expected Shortfall)")
print("=" * 40)
for level, value in cvar_results.items():
    print(f"{level} CVaR: ${abs(value):,.0f} ({abs(value/portfolio.nav)*100:.2f}%)")

In [ ]:
# Visualize VaR
returns = prices_df.pct_change().dropna()
portfolio_returns = (returns * pd.Series(portfolio.weights)).sum(axis=1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Return distribution
axes[0].hist(portfolio_returns, bins=50, alpha=0.7, edgecolor='black')
var_95_pct = np.percentile(portfolio_returns, 5)
var_99_pct = np.percentile(portfolio_returns, 1)
axes[0].axvline(var_95_pct, color='orange', linestyle='--', label=f'95% VaR: {var_95_pct:.2%}')
axes[0].axvline(var_99_pct, color='red', linestyle='--', label=f'99% VaR: {var_99_pct:.2%}')
axes[0].set_title('Portfolio Return Distribution')
axes[0].set_xlabel('Daily Return')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# Risk comparison bar chart
metrics = ['95% VaR', '99% VaR', '95% CVaR', '99% CVaR']
values = [
    abs(var_results['95%']),
    abs(var_results['99%']),
    abs(cvar_results['95%']),
    abs(cvar_results['99%']),
]
colors = ['#3498db', '#2980b9', '#e74c3c', '#c0392b']
axes[1].bar(metrics, values, color=colors)
axes[1].set_title('Risk Metrics Comparison')
axes[1].set_ylabel('Dollar Amount')
for i, v in enumerate(values):
    axes[1].text(i, v + 1000, f'${v:,.0f}', ha='center')

plt.tight_layout()
plt.show()

## 4. Greeks

In [ ]:
# Compute Greeks
greeks = risk_adapter.compute_greeks(portfolio, prices_df)

print("Portfolio Greeks")
print("=" * 40)
for name, value in greeks.items():
    if value is not None:
        print(f"{name.capitalize()}: {value:.4f}")
    else:
        print(f"{name.capitalize()}: N/A (not applicable for equity)")

## 5. Stress Testing

In [ ]:
from src.core.domain.stress_scenario import StressScenario

# Define stress scenarios
scenarios = [
    StressScenario(
        name='2008 Financial Crisis',
        shocks={'equity_all': -0.40},
        description='40% equity market decline',
        date_calibrated=date(2008, 10, 1),
    ),
    StressScenario(
        name='COVID Crash (Mar 2020)',
        shocks={'equity_all': -0.35},
        description='35% rapid equity decline',
        date_calibrated=date(2020, 3, 1),
    ),
    StressScenario(
        name='Tech Correction',
        shocks={'AAPL': -0.25, 'MSFT': -0.25, 'GOOGL': -0.30, 'META': -0.35},
        description='Tech sector specific correction',
        date_calibrated=date(2022, 1, 1),
    ),
    StressScenario(
        name='Mild Pullback',
        shocks={'equity_all': -0.10},
        description='10% market pullback',
        date_calibrated=date.today(),
    ),
]

# Run stress tests
stress_results = risk_adapter.stress_test(
    portfolio=portfolio,
    market_data=prices_df,
    scenarios=scenarios,
)

print("Stress Test Results")
print("=" * 60)
print(f"{'Scenario':<25} {'P&L':>15} {'% Change':>12}")
print("-" * 60)
for _, row in stress_results.iterrows():
    print(f"{row['scenario']:<25} ${row['pnl']:>14,.0f} {row['pct_change']*100:>11.2f}%")

In [ ]:
# Visualize stress test results
fig, ax = plt.subplots(figsize=(10, 6))

scenarios_names = stress_results['scenario'].tolist()
pnl_values = stress_results['pnl'].tolist()

colors = ['#c0392b' if v < 0 else '#27ae60' for v in pnl_values]
bars = ax.barh(scenarios_names, pnl_values, color=colors)

ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_xlabel('P&L ($)')
ax.set_title('Stress Test P&L by Scenario')

# Add value labels
for bar, val in zip(bars, pnl_values):
    ax.text(val, bar.get_y() + bar.get_height()/2, 
            f'${val:,.0f}', va='center', ha='left' if val > 0 else 'right')

plt.tight_layout()
plt.show()

## 6. Risk Summary

In [ ]:
# Create risk summary dashboard
print("\n" + "=" * 60)
print("PORTFOLIO RISK SUMMARY")
print("=" * 60)
print(f"\nPortfolio: {portfolio.metadata.strategy_name}")
print(f"NAV: ${portfolio.nav:,.0f}")
print(f"As of: {portfolio.as_of_date}")

print("\n--- Value at Risk ---")
print(f"1-Day 95% VaR: ${abs(var_results['95%']):,.0f} ({abs(var_results['95%']/portfolio.nav)*100:.2f}%)")
print(f"1-Day 99% VaR: ${abs(var_results['99%']):,.0f} ({abs(var_results['99%']/portfolio.nav)*100:.2f}%)")

print("\n--- Expected Shortfall ---")
print(f"1-Day 95% CVaR: ${abs(cvar_results['95%']):,.0f} ({abs(cvar_results['95%']/portfolio.nav)*100:.2f}%)")
print(f"1-Day 99% CVaR: ${abs(cvar_results['99%']):,.0f} ({abs(cvar_results['99%']/portfolio.nav)*100:.2f}%)")

print("\n--- Worst Stress Scenario ---")
worst = stress_results.loc[stress_results['pnl'].idxmin()]
print(f"Scenario: {worst['scenario']}")
print(f"Impact: ${worst['pnl']:,.0f} ({worst['pct_change']*100:.2f}%)")